In [177]:
import psycopg2
import clickhouse_connect
import pandas as pd
import os
import sys
import re
import numpy as np


# Подключение к PostgreSQL
def conn_pg():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="dbpg",
            user="user",
            password="password",
            port=5432
        )
        # print("Connected to PostgreSQL")
        # cur = conn.cursor()
        # cur.execute(query)
        # output = cur.fetchall()
        # print("PostgreSQL version:", db_version)
        # if output:
        #     return
        # else:
        #     print("No output from the query")
        return conn    
    except Exception as e:
        print(f"Error connecting to PostgreSQL: {e}")
    
def conn_pg_close(conn):
    if conn:
        cur = conn.cursor()
        cur.close()
        conn.close()
        # print("PostgreSQL connection closed")


# Подключение к ClickHouse
def conn_ch():
    try:
        client = clickhouse_connect.get_client(
            host='localhost',
            username='user',
            password='password',
            database='dbch',
            port=8123
        )
        # print("Connected to ClickHouse")
        # version = client.command('SELECT version()')
        # print("ClickHouse version:", version)
        return client
    except Exception as e:
        print(f"Error connecting to ClickHouse: {e}")

        # print("Connected to ClickHouse")
        # version = client.command('SELECT version()')

        # print("ClickHouse version:", version)


# import tools
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # for subfolder

# find tools in parent dir
if os.path.isdir(os.path.join(parent_dir, 'tools')):
    # Add parent directory to sys.path if found
    sys.path.append(parent_dir)
    
else:
    # for run in spark
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../airflow/airflow_data"))
    
    # Add parent directory to sys.path
    sys.path.append(parent_dir)


from tools import pd_tools
from tools.paths import Paths
from tools.db_tools import DbTools


# import paths
root_path = '..' # for local folder
paths = Paths(root_path)
data_path = paths.data_path
tmp_path = paths.tmp_path
prod_data_path = paths.prod_data_path
dev_data_path = paths.dev_data_path
prod_db = paths.prod_db
dev_db = paths.dev_db
tmp_db = paths.tmp_db
tmp_data_path = paths.tmp_data_path

# db_tools = DbTools(data_path, tmp_path, client)

## Load data and analyze it

In [2]:
fc_reg_path = f'{data_path}/film_cert_registry'

In [3]:
# write files to df dict

fc_reg_dict = pd_tools.dict_dfs(fc_reg_path)

/home/pmu/projects/tools/pd_tools.py:23: DtypeWarning: Columns (2,13,18,19,20,32,35,37,40,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_dfs[name] = pd.read_csv(path + '/' + file)
/home/pmu/projects/tools/pd_tools.py:23: DtypeWarning: Columns (2,12,13,14,17,19,20,32,35,36,37,40,44) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_dfs[name] = pd.read_csv(path + '/' + file)
/home/pmu/projects/tools/pd_tools.py:23: DtypeWarning: Columns (3,5,7,8,11,20,23,28,38,39,40,42,43,44,47,51) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_dfs[name] = pd.read_csv(path + '/' + file)
/home/pmu/projects/tools/pd_tools.py:23: DtypeWarning: Columns (3,5,7,8,11,20,23,28,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  dict_dfs[name] = pd.read_csv(path + '/' + file)
/home/pmu/projects/tools/pd_tools.py:23: DtypeWarning: Columns (2,13,18,19,20,32,35,37,41,44,45,46) h

In [4]:
file_names = list(fc_reg_dict.keys())  # Convert keys to a list for easier manipulation  
file_names

['data-6-structure-3',
 'data-7-structure-4',
 'data-2-structure-1',
 'data-1-structure-1',
 'data-4-structure-2',
 'data-3-structure-2',
 'data-9-structure-4',
 'data-8-structure-4']

## Rename common columns and create aggregated dataframe

In [5]:
# create list of columns for each df
columns = {}
for file_name in file_names:
    print(f'Processing: {file_name}')
    df = fc_reg_dict[file_name]
    columns[file_name] = list(df.columns)

Processing: data-6-structure-3
Processing: data-7-structure-4
Processing: data-2-structure-1
Processing: data-1-structure-1
Processing: data-4-structure-2
Processing: data-3-structure-2
Processing: data-9-structure-4
Processing: data-8-structure-4


In [6]:
# find common columns in all df
list_all_df_cols = []
for key, value in columns.items():
    list_all_df_cols.append(set(value))
    
common_columns = set.intersection(*list_all_df_cols)
common_columns

{'Аннотация',
 'Возрастная категория',
 'Год производства',
 'Дата регистрации удостоверения',
 'Единица измерения',
 'Идентификатор записи реестра',
 'Категория',
 'Количество серий',
 'Композитор',
 'Название фильма',
 'Номер удостоверения',
 'Объем носителя',
 'Оператор',
 'Порядковый номер переоформления/продления',
 'Права на использование фильма',
 'Права проката переданы',
 'Примечание',
 'Продюсер',
 'Режиссер',
 'Страна производства',
 'Студия-производитель',
 'Сценарист',
 'Формат кадра',
 'Формат носителя',
 'Художник',
 'Цвет',
 'Язык оригинала',
 'дата обновления объекта',
 'дата создания объекта'}

In [7]:
duration_columns = [
    'Продолжительность демонстрации, часы',
    'Продолжительность демонстрации, минуты',
    'Продолжительность показа, минуты', 
    'Продолжительность показа, часы'
]

In [ ]:
column_rename_map = {
    'Идентификатор записи реестра': 'registry_record_id',
    'Название фильма': 'film_title',
    'Страна производства': 'country_of_production',
    'Студия-производитель': 'production_studio',
    'Год производства': 'year_of_production',
    'Дата регистрации удостоверения': 'certificate_registration_date',
    'Категория': 'category',
    'Права на использование фильма': 'film_usage_rights',
    'Номер удостоверения': 'certificate_number',
    'Возрастная категория': 'age_rating',
    'Продолжительность демонстрации, часы': 'duration_hours',
    'Продолжительность демонстрации, минуты': 'duration_minutes',
    'Продолжительность показа, часы': 'duration_hours', # same as demonstration_duration_hours
    'Продолжительность показа, минуты': 'duration_minutes',  # same as demonstration_duration_minutes
    'Аннотация': 'synopsis',
    'Количество серий': 'number_of_episodes',    
    'Объем носителя': 'media_size',
    'Единица измерения': 'unit_of_measurement',    
    'Порядковый номер переоформления/продления': 'amendment_extension_number',
    'Права проката переданы': 'distribution_rights_transferred',
    'Примечание': 'note',
    'Режиссер': 'director',
    'Продюсер': 'producer',
    'Сценарист': 'screenwriter',
    'Оператор': 'cinematographer',
    'Художник': 'art_director',
    'Композитор': 'composer', 
    'Студия-производитель': 'production_studio',    
    'Формат кадра': 'frame_format',
    'Формат носителя': 'media_format',
    'Художник': 'art_director',
    'Цвет': 'color',
    'Язык оригинала': 'original_language',
    'дата обновления объекта': 'object_update_date',
    'дата создания объекта': 'object_creation_date',
}

# cretae list of columns for new dict_df
new_df_columns = list(column_rename_map.values())

new_df_columns = pd.Series(new_df_columns).drop_duplicates().to_list()
new_df_columns

['registry_record_id',
 'film_title',
 'country_of_production',
 'production_studio',
 'year_of_production',
 'certificate_registration_date',
 'category',
 'film_usage_rights',
 'certificate_number',
 'age_rating',
 'duration_hours',
 'duration_minutes',
 'unit_of_measurement',
 'synopsis',
 'number_of_episodes',
 'media_size',
 'amendment_extension_number',
 'distribution_rights_transferred',
 'note',
 'director',
 'producer',
 'screenwriter',
 'cinematographer',
 'art_director',
 'composer',
 'frame_format',
 'media_format',
 'color',
 'original_language',
 'object_update_date',
 'object_creation_date']

In [205]:
# create new dict_df with only renamed columns
fc_reg_dict_new = {}
for file_name in file_names:
    print(f'Processing: {file_name}')
    df_old = fc_reg_dict[file_name]
    df_new = df_old.rename(columns=column_rename_map)
    df_new = df_new.copy()[new_df_columns]
    fc_reg_dict_new[file_name] = df_new  # Update the null dictionary with df has renamed columns only

# create df with all df from dict
fc_reg_df = pd.concat(fc_reg_dict_new.values(), ignore_index=True)
fc_reg_df.head()

Processing: data-6-structure-3
Processing: data-7-structure-4
Processing: data-2-structure-1
Processing: data-1-structure-1
Processing: data-4-structure-2
Processing: data-3-structure-2
Processing: data-9-structure-4
Processing: data-8-structure-4


,registry_record_id,film_title,country_of_production,production_studio,year_of_production,certificate_registration_date,category,film_usage_rights,certificate_number,age_rating,...,screenwriter,cinematographer,art_director,composer,frame_format,media_format,color,original_language,object_update_date,object_creation_date
0,2157027,Почти знаменит,США,Коламбиа Пикчерз,2000,2016-02-09T12:00:00.000Z,Видео,"ЗАО ""ФЕЛИСТА""",221006316,«18+» запрещено для детей,...,Кемерон Кроу,NaN,NaN,Джон Бонхэм,NaN,NaN,Цветной,NaN,2018-04-09T09:12:02+03:00,2017-07-05T14:10:11+03:00
1,2157028,Открытое окно.,США,"Кард Интертейнмент, Имэйдж Интертейнмент",2006,2014-06-30T12:00:00.000Z,Видео,"ООО ""Люксор Дистрибьюшн""",221027814,«16+» для детей старше 16 лет,...,Миа Голдмен,NaN,NaN,Клифф Эйдельман,NaN,NaN,Цветной,NaN,2018-04-09T09:12:02+03:00,2017-07-05T14:10:11+03:00
2,2157029,Открытый простор,США,"Тачстоун Пикчерз, Кобальт Пикчерз, Бикон Пикче...",2003,2015-11-27T12:00:00.000Z,Кино,"ООО ""Дистрибуционная кинокомпания ""Парадиз""",221048915,«18+» запрещено для детей,...,"Лоран Пэйн, Крейг Сторпер",Джеймс М.Муро,Гэй С.Бакли,Майкл Кэмен,ш/э,NaN,Цветной,NaN,2018-04-09T09:12:02+03:00,2017-07-05T14:10:11+03:00
3,2157030,Особо важное задание,СССР,"Киностудия ""Мосфильм""",1980,2016-09-13T12:00:00.000Z,Кино,"ФГУП ""Киноконцерн ""Мосфильм""",111013716,«6+» для детей старше шести лет,...,"Б.Добродеев, П.Попогребский",И.Черных,С.Валюшок,Е.Птичкин,ш/э,NaN,Цветной,NaN,2018-04-09T09:12:02+03:00,2017-07-05T14:10:11+03:00
4,2157031,Особо опасен,США,"Юниверсал Пикчерз, Кикстарт Продакшнз, Марк Пл...",2008,2016-10-10T12:00:00.000Z,Кино,"ООО ""НД Плэй""",221038416,«18+» запрещено для детей,...,"Майкл Брандт, Крис Морган, Дерек Хаас",Митчелл Амундсен,Джон Майр,Дэнни Эльфман,ш/э,NaN,Цветной,NaN,2019-05-20T11:24:32+03:00,2017-07-05T14:10:11+03:00


In [10]:
pd_tools.df_info(fc_reg_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745664 entries, 0 to 745663
Data columns (total 31 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   registry_record_id               745664 non-null  int64  
 1   film_title                       745664 non-null  object 
 2   country_of_production            745141 non-null  object 
 3   production_studio                744662 non-null  object 
 4   year_of_production               743963 non-null  object 
 5   certificate_registration_date    745550 non-null  object 
 6   category                         745664 non-null  object 
 7   film_usage_rights                732182 non-null  object 
 8   certificate_number               745664 non-null  object 
 9   age_rating                       745446 non-null  object 
 10  demonstration_duration_hours     616376 non-null  float64
 11  demonstration_duration_minutes   726036 non-null  object 
 12  un

'First 5 rows in df'

,registry_record_id,film_title,country_of_production,production_studio,year_of_production,certificate_registration_date,category,film_usage_rights,certificate_number,age_rating,...,screenwriter,cinematographer,art_director,composer,frame_format,media_format,color,original_language,object_update_date,object_creation_date
0,2157027,Почти знаменит,США,Коламбиа Пикчерз,2000,2016-02-09T12:00:00.000Z,Видео,"ЗАО ""ФЕЛИСТА""",221006316,«18+» запрещено для детей,...,Кемерон Кроу,NaN,NaN,Джон Бонхэм,NaN,NaN,Цветной,NaN,2018-04-09T09:12:02+03:00,2017-07-05T14:10:11+03:00
1,2157028,Открытое окно.,США,"Кард Интертейнмент, Имэйдж Интертейнмент",2006,2014-06-30T12:00:00.000Z,Видео,"ООО ""Люксор Дистрибьюшн""",221027814,«16+» для детей старше 16 лет,...,Миа Голдмен,NaN,NaN,Клифф Эйдельман,NaN,NaN,Цветной,NaN,2018-04-09T09:12:02+03:00,2017-07-05T14:10:11+03:00
2,2157029,Открытый простор,США,"Тачстоун Пикчерз, Кобальт Пикчерз, Бикон Пикче...",2003,2015-11-27T12:00:00.000Z,Кино,"ООО ""Дистрибуционная кинокомпания ""Парадиз""",221048915,«18+» запрещено для детей,...,"Лоран Пэйн, Крейг Сторпер",Джеймс М.Муро,Гэй С.Бакли,Майкл Кэмен,ш/э,NaN,Цветной,NaN,2018-04-09T09:12:02+03:00,2017-07-05T14:10:11+03:00
3,2157030,Особо важное задание,СССР,"Киностудия ""Мосфильм""",1980,2016-09-13T12:00:00.000Z,Кино,"ФГУП ""Киноконцерн ""Мосфильм""",111013716,«6+» для детей старше шести лет,...,"Б.Добродеев, П.Попогребский",И.Черных,С.Валюшок,Е.Птичкин,ш/э,NaN,Цветной,NaN,2018-04-09T09:12:02+03:00,2017-07-05T14:10:11+03:00
4,2157031,Особо опасен,США,"Юниверсал Пикчерз, Кикстарт Продакшнз, Марк Пл...",2008,2016-10-10T12:00:00.000Z,Кино,"ООО ""НД Плэй""",221038416,«18+» запрещено для детей,...,"Майкл Брандт, Крис Морган, Дерек Хаас",Митчелл Амундсен,Джон Майр,Дэнни Эльфман,ш/э,NaN,Цветной,NaN,2019-05-20T11:24:32+03:00,2017-07-05T14:10:11+03:00


,duplicates_full
duplicates_full,423895


,registry_record_id,film_title,country_of_production,production_studio,year_of_production,certificate_registration_date,category,film_usage_rights,certificate_number,age_rating,...,screenwriter,cinematographer,art_director,composer,frame_format,media_format,color,original_language,object_update_date,object_creation_date
duplicates_by_cols,641157,658956,741908,703854,742690,733695,745659,719251,603415,745409,...,600139,384331,259158,450996,81121,59822,745560,25,715825,724660


...

""
nones


""
NA placeholder


""
null placeholder


""
N/A placeholder


,<class 'str'>,<class 'int'>,<class 'float'>
certificate_number,536071,209593.0,NaN
demonstration_duration_minutes,29066,NaN,696970.0
number_of_episodes,19312,NaN,278654.0
media_size,27086,NaN,3.0


## Data transform

In [224]:
def not_same_types(df: pd.DataFrame, col_name: str) -> pd.Series | None:
    """
    If there are multiple dtypes in col, return a col_name with counts of each dtype.
    Null values are ignored in the check.
    If all dtypes are the same, return None.    
    """
    non_null_col = df[col_name][df[col_name].notnull()]
    col_types = non_null_col.apply(type).value_counts()
    if len(col_types) > 1:
        return col_types
    else:
        print(f'Column {col_name} has only one type: {col_types.index[0]}')
        return None  
    
def str_class(df: pd.DataFrame, col_name: str) -> pd.DataFrame:
    '''
    Classifies values in col_name as:
    - is_digit: all digits
    - is_alpha: all alphabetic (A-Z or Cyrillic)
    - is_empty: empty string after strip
    - has_special_symbols: any character excluding . , letters, digits, space
    - has_letters: contains at least one letter
    - has_dots: contains "."
    - has_commas: contains ","
    - has_spaces: contains space
    - nans_nulls: null or NaN
    - etc: does not match any above
    '''
    raw_series = df[col_name]
    series = raw_series.astype(str).str.strip()

    is_digit = series.apply(lambda x: x.isdigit())
    # NaN becomes the string 'nan' when cast to str.
    is_alpha = series.apply(lambda x: x.isalpha() if x and (x != 'nan') else False)
    is_empty = series.apply(lambda x: x == '')

    has_letters = series.apply(lambda x: any(c.isalpha() for c in x) if x and (x != 'nan') else False)
    has_special_symbols = series.apply(
        lambda x: bool(re.search(r'[^a-zA-Zа-яА-ЯёЁ0-9\s.,]', x))
    )
    has_dots = series.apply(lambda x: '.' in x)
    has_commas = series.apply(lambda x: ',' in x)
    has_spaces = series.apply(lambda x: ' ' in x)

    nans_nulls = raw_series.isnull()

    # etc = does not match any of the above and is not null
    etc = ~(is_digit | is_alpha | is_empty | has_special_symbols |
            has_letters | has_dots | has_commas | has_spaces | nans_nulls)

    results_df = pd.DataFrame({
        'is_digit': is_digit,
        'is_alpha': is_alpha,
        'is_empty': is_empty,
        'has_special_symbols': has_special_symbols,
        'has_letters': has_letters,
        'has_dots': has_dots,
        'has_commas': has_commas,
        'has_spaces': has_spaces,
        'nans_nulls': nans_nulls,
        'etc': etc
    })

    summary = results_df.sum().reset_index()
    summary.columns = ['text_property', 'count']

   

    return summary


In [247]:
str_class(fc_reg_df, 'certificate_number')

,text_property,count
0,is_digit,731344
1,is_alpha,12
2,is_empty,0
3,has_special_symbols,54
4,has_letters,14266
5,has_dots,0
6,has_commas,0
7,has_spaces,64
8,nans_nulls,0
9,etc,0


In [248]:
# find rows where certificate_number is not only digits
mask_not_digit = ~fc_reg_df['certificate_number'].astype(str).str.strip().str.isdigit()
fc_reg_df.loc[mask_not_digit, 'certificate_number']


7936             нет
16864     121012208А
17641     221113209_
25246     2210-45210
27015      213-11610
             ...    
730286    221123801Д
730307    221008000Д
730382    221050601Д
730383    221098103Д
730384    221104600Д
Name: certificate_number, Length: 14320, dtype: object

In [249]:
# change to str and removes any leading and trailing whitespace characters from the string values.
fc_reg_df['certificate_number'] = fc_reg_df['certificate_number'].astype(str).str.strip()
not_same_types(fc_reg_df, 'certificate_number')

Column certificate_number has only one type: <class 'str'>


In [208]:
not_same_types(fc_reg_df, 'duration_minutes')

duration_minutes
<class 'float'>    696970
<class 'str'>       29066
Name: count, dtype: int64

In [227]:
str_class(fc_reg_df, 'duration_minutes')

,text_property,count
0,is_digit,29057
1,is_alpha,0
2,is_empty,0
3,has_special_symbols,9
4,has_letters,0
5,has_dots,696970
6,has_commas,0
7,has_spaces,0
8,nans_nulls,19628
9,etc,0


In [210]:
# find rows with special characters in duration_minutes
has_spec_sym = fc_reg_df['duration_minutes'].astype(str).apply(lambda x: bool(re.search(r'[^a-zA-Zа-яА-ЯёЁ0-9\s.,]', x)))
fc_reg_df[has_spec_sym]['duration_minutes']

188562    58/59
190277      7/8
191417    28/25
638127    58/59
639842      7/8
640981    28/25
741296    58/59
743010      7/8
744152    28/25
Name: duration_minutes, dtype: object

In [230]:
fc_reg_df['duration_minutes'] = (fc_reg_df['duration_minutes']
                .astype(str)                 # Convert to string (NaN -> 'nan')
                .str.split('/')              # Split by '/'
                .str[0]                     # Take first part
                .str.strip()                # Strip whitespace
                .replace('nan', '-1')       # Replace 'nan' string with '-1'
                .astype(float)              # Convert to float
                .astype(int)                # Convert to int
            )

In [232]:
not_same_types(fc_reg_df, 'duration_minutes')

Column duration_minutes has only one type: <class 'int'>


In [ ]:
not_same_types(fc_reg_df, 'number_of_episodes')

number_of_episodes
<class 'float'>    278654
<class 'str'>       19312
Name: count, dtype: int64

In [236]:
str_class(fc_reg_df, 'number_of_episodes')

,text_property,count
0,is_digit,19309
1,is_alpha,0
2,is_empty,0
3,has_special_symbols,3
4,has_letters,3
5,has_dots,278654
6,has_commas,0
7,has_spaces,3
8,nans_nulls,447698
9,etc,0


In [237]:
# find special characters in number_of_episodes
has_spec_sym = fc_reg_df['number_of_episodes'].astype(str).apply(lambda x: bool(re.search(r'[^a-zA-Zа-яА-ЯёЁ0-9\s.,]', x)))
fc_reg_df[has_spec_sym]['number_of_episodes']

190215    5 (4-8 из 8)
639780    5 (4-8 из 8)
742948    5 (4-8 из 8)
Name: number_of_episodes, dtype: object

In [243]:
# find space in number_of_episodes
has_space = fc_reg_df['number_of_episodes'].astype(str).apply(lambda x: ' ' in x)
fc_reg_df[has_space]['number_of_episodes']

190215    5 (4-8 из 8)
639780    5 (4-8 из 8)
742948    5 (4-8 из 8)
Name: number_of_episodes, dtype: object

In [246]:
# remove everything after '(' in number_of_episodes
# and replace 'nan' with '-1', strip whitespace, convert to float and then to int
fc_reg_df['number_of_episodes'] = fc_reg_df['number_of_episodes'].astype(str).str.split('(').str[0].replace('nan', '-1').str.strip().astype(float).astype(int)
fc_reg_df['number_of_episodes'].head()

0   -1
1   -1
2    1
3    2
4    1
Name: number_of_episodes, dtype: int64

In [252]:
not_same_types(fc_reg_df,'media_size')


media_size
<class 'str'>      27086
<class 'float'>        3
Name: count, dtype: int64

In [253]:
str_class(fc_reg_df, 'media_size')

,text_property,count
0,is_digit,16871
1,is_alpha,0
2,is_empty,0
3,has_special_symbols,8
4,has_letters,122
5,has_dots,2105
6,has_commas,8036
7,has_spaces,119
8,nans_nulls,718575
9,etc,0


In [276]:
fc_reg_df['media_size'] = fc_reg_df['media_size'].astype(str).str.replace(',', '.')
fc_reg_df[fc_reg_df['media_size'] != 'nan']['media_size'].head()

14472      51.2
14846    889Гб.
14914       104
15770        46
15923         1
Name: media_size, dtype: object

In [277]:
# find special characters in number_of_episodes
has_spec_sym = fc_reg_df['media_size'].astype(str).apply(lambda x: bool(re.search(r'[^a-zA-Zа-яА-ЯёЁ0-9\s.,]', x)))
fc_reg_df[has_spec_sym]['media_size']

86809      100/170 
179729     100/170 
190936    3400/5585
451545     100/170 
629293     100/170 
640500    3400/5585
732462     100/170 
743668    3400/5585
Name: media_size, dtype: object

In [281]:
# find letters in number_of_episodes
has_letters = fc_reg_df['media_size'].astype(str).apply(lambda x: any(c.isalpha() for c in x) if x and (x != 'nan') else False)
fc_reg_df[has_letters][['media_size', 'unit_of_measurement']]

,media_size,unit_of_measurement
14846,889Гб.,NaN
87602,1.3 Тб,NaN
89672,664 Гб,NaN
89738,58 Гб,NaN
89853,119 Гб,NaN
...,...,...
745084,3.55 ГБ,NaN
745085,4.3 Гб,NaN
745087,4.7 ГБ,NaN
745088,4.7 ГБ,NaN


In [ ]:
units_list = ['ГБ', 'ГБ.', 'ТБ', 'ТБ.','Гб', 'Гб.', 'Тб', 'Тб.']

# find units from units_list in media_size and split by unit
for unit in units_list:
    mask = fc_reg_df['media_size'].astype(str).str.contains(unit, na=False)
    if mask.any():
        fc_reg_df.loc[mask, 'media_size'] = fc_reg_df.loc[mask, 'media_size'].str.split(unit).str[0].str.strip()
        fc_reg_df.loc[mask, 'unit_of_measurement'] = unit




0           [-1]
1           [-1]
2           [-1]
3           [-1]
4           [-1]
           ...  
745659      [-1]
745660      [-1]
745661      [-1]
745662    [2.41]
745663    [2.41]
Name: media_size, Length: 745664, dtype: object

In [280]:
fc_reg_df['unit_of_measurement'].value_counts()

unit_of_measurement
Гб    15200
Тб    11480
Мб       84
ГБ       30
ТБ       12
Gb        4
Name: count, dtype: int64

In [284]:
(fc_reg_df['media_size']
                .astype(str)                 # Convert to string (NaN -> 'nan')
                .str.split('/')              # Split by '/'
                .str[0]                     # Take first part
                .str.strip()                # Strip whitespace
                .replace('nan', '-1')       # Replace 'nan' string with '-1'
                # .astype(float)              # Convert to float
                # .astype(int)                # Convert to int
            )

0           -1
1           -1
2           -1
3           -1
4           -1
          ... 
745659      -1
745660      -1
745661      -1
745662    2.41
745663    2.41
Name: media_size, Length: 745664, dtype: object

## PostgreSQL DDL

In [6]:
# Создание курсора
conn = conn_pg()

cur = conn.cursor()

# Создание таблицы
cur.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    position VARCHAR(100),
    salary NUMERIC
);
''')
print("Table 'employees' created")

# Вставка данных в таблицу
cur.execute('''
INSERT INTO employees (name, position, salary)
VALUES ('Alice', 'Manager', 70000),
        ('Bob', 'Developer', 60000),
        ('Charlie', 'Designer', 50000);
''')
print("Data inserted into 'employees' table")

# Сохранение изменений
conn.commit()


Table 'employees' created
Data inserted into 'employees' table


In [7]:
# Закрытие курсора и соединения
conn_pg_close(conn)

In [9]:
try:
    # Создание таблицы
    client.command('''
    CREATE TABLE IF NOT EXISTS dbch.employees (
        id UInt32,
        name String,
        position String,
        salary Float32
    ) ENGINE = MergeTree()
    ORDER BY id;
    ''')
    print("Table 'employees' created in ClickHouse")

except Exception as e:
    print(f"Error connecting to ClickHouse: {e}")


Table 'employees' created in ClickHouse


In [15]:
client.query('select * from dbch.employees')

In [16]:
# # Correct way to insert rows
# data = [
#     (1, 'Alice', 'Manager', 70000.0),
#     (2, 'Bob', 'Developer', 60000.0),
#     (3, 'Charlie', 'Designer', 50000.0)
# ]

# client.insert('dbch.employees', data, column_names=['id', 'name', 'position', 'salary'])